In [1]:
from utils import Logger,adjust_learning_rate,CreateLogger,create_cosine_learning_schedule,encode_test_label,set_seed
from model import Resnet_with_uncertainty_derm, MLP_with_uncertainty#不确定度
from dependency import *
from torch import optim
from torchcontrib.optim import SWA
from dataloader import generate_dataloader
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error # 均方误差
from tensorflow.keras.utils import to_categorical
import pandas as pd
from dependency import *
%matplotlib notebook

In [2]:
from model import Resnet_with_uncertainty_derm,MyCNN,ResNet50_uncertainty,MLP_with_uncertainty
from second_stage_fusion_utils_concat2 import predict,predict3
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error 
import numpy as np
from sklearn.linear_model import BayesianRidge, LinearRegression, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor  
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib notebook

100%|████████████████████████████████████████████████████████████████████████████████| 395/395 [00:04<00:00, 89.63it/s]


In [3]:
import torch.nn.functional as F
from tqdm import tqdm
import torch
import numpy as np
from sklearn.metrics import roc_auc_score,accuracy_score, roc_curve,auc,average_precision_score,precision_recall_curve
import cv2
from dependency import *
import pandas as pd
from utils import encode_test_label,Logger,encode_meta_choosed_label,encode_meta_label
from tqdm import tqdm_notebook  

test_index_df = pd.read_csv(test_index_path)
train_index_df = pd.read_csv(train_index_path)
val_index_df = pd.read_csv(val_index_path)

train_index_list = list(train_index_df['indexes'])
val_index_list = list(val_index_df['indexes'])
test_index_list = list(test_index_df['indexes'])

train_index_list_1 = train_index_list[0:206]
train_index_list_2 = train_index_list[206:]

df = pd.read_csv(img_info_path)

In [4]:
def get_label_list(image_index_list):
    diag_label_list = []
    pn_label_list = []
    str_label_list = []
    pig_label_list = []
    rs_label_list = []
    dag_label_list = []
    bwv_label_list = []
    vs_label_list = []
    meta_list = []
    meta_list_one_hot = []
    
    img_feature = []
    img_hf_feature = []
    img_vf_feature = []
    img_vhf_feature = []

    from sklearn.decomposition import PCA

    from tqdm import tqdm_notebook,tqdm
    for index_num in tqdm(image_index_list):
    #index_num = test_index_list[100]
        img_info = df[index_num:index_num+1]
        clinic_path = img_info['clinic']
        dermoscopy_path = img_info['derm']
        source_dir = '../release_v0/release_v0/images/'
        clinic_img = cv2.imread(source_dir+clinic_path[index_num])
        dermoscopy_img = cv2.imread(source_dir+dermoscopy_path[index_num])
        '''这里对于metadata需要变更'''
        meta_vector_one_hot,meta_vector = encode_meta_choosed_label(img_info,index_num)
        
        [diagnosis_label,pigment_network_label,streaks_label,pigmentation_label,regression_structures_label,
         dots_and_globules_label,blue_whitish_veil_label, vascular_structures_label],[diagnosis_label_one_hot,pigment_network_label_one_hot,
        streaks_label_one_hot,pigmentation_label_one_hot,regression_structures_label_one_hot,
        dots_and_globules_label_one_hot,blue_whitish_veil_label_one_hot, vascular_structures_label_one_hot] = encode_test_label(img_info,index_num)

        diag_label_list.append(diagnosis_label)
        pn_label_list.append(pigment_network_label)
        str_label_list.append(streaks_label)
        pig_label_list.append(pigmentation_label)
        rs_label_list.append(regression_structures_label)
        dag_label_list.append(dots_and_globules_label)
        bwv_label_list.append(blue_whitish_veil_label)
        vs_label_list.append(vascular_structures_label)
        meta_list.append(meta_vector)
        meta_list_one_hot.append(meta_vector_one_hot)



    label_dict ={'diag':diag_label_list,
                 'pn':pn_label_list,
                 'str':str_label_list,
                 'pig':pig_label_list,
                 'rs':rs_label_list,
                 'dag':dag_label_list,
                 'bwv':bwv_label_list,
                 'vs':vs_label_list}
    
    return label_dict,meta_list,meta_list_one_hot

# multi-modality with uncertainty(derm and metadata) fusion here

In [9]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
seed = 0
mode = 'multimodal'
model_name = 'Resnet_derm'
import random
random.seed(seed)
TTA = 6
img_type="derm"
size = 229
data_mode = 'Normal'

#change i here!
i = 4

p1_acc_list = []
p2_acc_list = []
p3_acc_list = []

mean_avg_acc_list = []

weight_file = './{}_{}_{}_weight_file/{}/checkpoint/best_mean_acc_model.pth'.format(mode,model_name,data_mode,i)
print(weight_file)

net = Resnet_with_uncertainty_derm(class_list).cuda()
net.load_state_dict(torch.load(weight_file))

print("P2--------------------------------on metadata")
weight_file2 = './{}_{}_{}_weight_file/{}/checkpoint/best_mean_acc_model.pth'.format("multimodal","MLP",data_mode,i)
print(weight_file2)

net2 =  MLP_with_uncertainty(class_list).cuda()

net2.load_state_dict(torch.load(weight_file2))
out_dir = './{}_{}_result/{}/'.format(mode,model_name,"P2")
(avg_acc,uncertainty_pred,
     [prob,pn_prob,str_prob,pig_prob,rs_prob,dag_prob,bwv_prob,vs_prob], 
     [nevu_label, bcc_label, mel_label, misc_label, sk_label],
     [nevu_prob ,bcc_prob ,  mel_prob ,misc_prob ,sk_prob],
     seven_point_feature_list,
     [gt,pn_gt,str_gt,pig_gt,rs_gt,dag_gt, bwv_gt,vs_gt] )= predict(net2,net,test_index_list,df,
                                                                    model_name,out_dir,mode,TTA,size,img_type,"metadata")
prob_2 = prob
pn_prob_2 = pn_prob
str_prob_2 = str_prob
pig_prob_2 = pig_prob
rs_prob_2 = rs_prob
dag_prob_2 = dag_prob
bwv_prob_2 = bwv_prob
vs_prob_2 = vs_prob
uncertainty_2 = uncertainty_pred
p2_list  = [prob_2,pn_prob_2,str_prob_2,pig_prob_2,rs_prob_2,dag_prob_2,bwv_prob_2,vs_prob_2,uncertainty_2]
print("P2 Done")

./multimodal_Resnet_derm_Normal_weight_file/4/checkpoint/best_mean_acc_model.pth
P2--------------------------------on metadata
./multimodal_MLP_Normal_weight_file/4/checkpoint/best_mean_acc_model.pth

--- [START IDENTIFIER] ----------------------------------------------------------------




100%|████████████████████████████████████████████████████████████████████████████████| 395/395 [00:24<00:00, 16.15it/s]

---------------

avg_acc : 0.7639240506329114

vs_acc : 0.7974683544303798

bwv_acc : 0.8911392405063291

dag_acc : 0.6734177215189874

rs_acc : 0.8

pig_acc : 0.7139240506329114

str_acc : 0.7316455696202532

pn_acc : 0.7417721518987341

diag_acc : 0.7620253164556962

---------------

nevu_auc: 0.928964300539643

bcc_auc: 0.9444261213720316

mel_auc: 0.9194786825621337

misc_auc: 0.9311971830985916

sk_auc: 0.9090145576707727

---------------

vs_asb_auc: 0.8567755006623549

vs_reg_auc: 0.8455931823278762

vs_irg_auc: 0.7649315068493151

---------------

bwv_asb_auc: 0.9306666666666666

bwv_prs_auc: 0.9306666666666665

---------------

dag_asb_auc: 0.7844745762711864

dag_reg_auc: 0.822584592792021

dag_irg_auc: 0.851500544238843

---------------

rs_asb_auc: 0.8640399556048834

rs_prs_auc: 0.8640399556048836

---------------

pig_asb_auc: 0.8433621858379393

pig_reg_auc: 0.8608909702209413

pig_irg_auc: 0.8671586715867158

---------------

str_asb_auc: 0.8576946935092766

str_reg_auc